# Лабораторная работа №2
## Выполнил студент группы БСТ1904 Костин Антон Алексеевич
### Задание №1

Реализовать методы поиска в соответствии с заданием. Организовать генерацию
начального набора случайных данных. Для всех вариантов добавить реализацию
добавления, поиска и удаления элементов. Оценить время работы каждого алгоритма
поиска и сравнить его со временем работы стандартной функции поиска, используемой в
выбранном языке программирования. 

|Бинарный поиск|Бинарное дерево|Фибоначчиев|Интерполяционный|
|------|------|------|------|

### Генерация матрицы

In [3]:
import numpy as np

def randomMatrix(min, max, rows, columns):
    return np.random.randint(min, max, (rows, columns))

### Бинарный поиск


In [4]:
class binarySearch:
    def __init__(self):
        self.data = []

    # Поиск индекса
    def search(self, key):
        min = 0
        max = len(self.data)
        while min < max:
            middle = (min + max) // 2
            if self.data[middle][0] < key:
                min = middle + 1
            else:
                max = middle
        return min    
       
    # Добавить элемент
    def __setitem__(self, key, value):
        index = self.search(key)
        if index < len(self.data) and self.data[index][0] == key:
            self.data[index] = (key, value)
        else:
            self.data.insert(index, (key, value))
    
    # Удалить элемент
    def __delitem__(self, key):
        index = self.search(key)
        self.data.pop(index)
    
    # Получить элемент
    def __getitem__(self, key):
        index = self.search(key)
        foundKey, val = self.data[index]
        if foundKey == key:
            return val

### Бинарное дерево
Значения меньше корневого находятся в левом поддереве, а большие - в правом

In [5]:
class binaryTree:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.left = self.right = None
        
class binaryTreeMap:
    root = None
    
    def insert(self, tree, key, value):
        if tree is None:
            return binaryTree(key, value)
        if tree.key > key:
            tree.left = self.insert(tree.left, key, value)
        elif tree.key < key:
            tree.right = self.insert(tree.right, key, value)
        else:
            tree.value = value
        return tree
            
    def search(self, tree, key):
        if tree is None or tree.key == key:
            return tree
        if tree.key > key:
            return self.search(tree.left, key)
        return self.search(tree.right, key)
        
    def __getitem__(self, key):
        tree = self.search(self.root, key)
        if tree is not None:
            return tree.value
        raise KeyError()
    
    def __setitem__(self, key, value):
        if self.root is None:
            self.root = self.insert(self.root, key, value)
        else: self.insert(self.root, key, value)

### Фибоначчиев поиск
В этом поиске анализируются элементы, находящиеся в позициях, равных числам Фибоначчи.  
Числа Фибоначчи получаются по следующему правилу: последующее число равно сумме двух предыдущих чисел.  
##### [1, 1, 2, 3, 5, 8, 13, 21, 34 ....]

In [ ]:
def fib(n):
    if len(fib_c) - 1 < n:
        fib_c.append(fib(n - 1) + fib(n - 2))
    return fib_c[n]

class fibonacciMap(BinarySearchMap):
    def search(self, key):
        m = 0 
        while fib(m) < len(self.data): 
            m += 1
        offset = 0
        while fib(m) > 1:
            i = min(offset + fib(m - 1), len(self.data) - 1)
            if key > self.data[i][0]:
                offset = i
            elif key == self.data[i][0]:
                return i
            m -= 1
        if len(self.data) and self.data[offset][0] < key:
            return offset + 1
        return 0

### Интерполяционный поиск

In [ ]:
def nearest_mid(input_list, lower_bound_index, upper_bound_index, search_value):
    return lower_bound_index + \
        (upper_bound_index - lower_bound_index) * \
        (search_value - input_list[lower_bound_index]) // \
        (input_list[upper_bound_index][0] - input_list[lower_bound_index][0])

class interpolateMap(BinarySearchMap):
    def interpolation_search(self, term):
        size_of_list = len(self.data) - 1

        index_of_first_element = 0
        index_of_last_element = size_of_list

        while index_of_first_element <= index_of_last_element:
            mid_point = nearest_mid(self.data, index_of_first_element, index_of_last_element, term)

            if mid_point > index_of_last_element or mid_point < index_of_first_element:
                return None

            if self.data[mid_point][0] == term:
                return mid_point

            if term > self.data[mid_point][0]:
                index_of_first_element = mid_point + 1
            else:
                index_of_last_element = mid_point - 1

        if index_of_first_element > index_of_last_element:
            return None

### Задание 2
|Простое рехэширование|Рехэширование с помощью псевдослучайных чисел|Метод цепочек|
|------|------|------|

### Простое рехэширование


In [9]:
class HashMap:
    def __init__(self):
        self.size = 0
        self.data = []
        self._resize()
    
    def _hash(self, key, i):
        return (hash(key) + i) % len(self.data)
        
    def _find(self, key):
        i = 0;
        index = self._hash(key, i);
        while self.data[index] is not None and self.data[index][0] != key:
            i += 1
            index = self._hash(key, i);
        return index;
    
    def _resize(self):
        temp = self.data
        self.data = [None] * (2*len(self.data) + 1)
        for item in temp:
            if item is not None:
                self.data[self._find(item[0])] = item
    
    def __setitem__(self, key, value):
        if self.size + 1 > len(self.data) // 2:
            self._resize()
        index = self._find(key)
        if self.data[index] is None:  
            self.size += 1
        self.data[index] = (key, value)
    
    def __getitem__(self, key):
        index = self._find(key)
        if self.data[index] is not None:
            return self.data[index][1]
        raise KeyError()

### Рехеширование с помощью всевдослучайных чисел

In [10]:
class RandomHashMap(HashMap):
    _rand_c = [5323]
    
    def _rand(self, i):
        if len(self._rand_c) - 1 < i:
            self._rand_c.append(self._rand(i - 1))
        return (123456789 * self._rand_c[i] + 987654321) % 65546
        
    def _hash(self, key, i):
        return (hash(key) + self._rand(i)) % len(self.data)

### Метод цепочек

In [ ]:
class ChainMap:
    def __init__(self):
        self.size = 0
        self.data = []
        self._resize()
    
    def _hash(self, key):
        return hash(key) % len(self.data)
    
    def _insert(self, index, item):
        if self.data[index] is None:
            self.data[index] = [item]
            return True
        else:
            for i, item_ in enumerate(self.data[index]):
                if item_[0] == item[0]:
                    self.data[index][i] = item
                    return False
            self.data[index].append(item)
            return True
    
    def _resize(self):
        temp = self.data
        self.data = [None] * (2*len(self.data) + 1)
        for bucket in temp:
            if bucket is not None:
                for key, value in bucket:
                    self._insert(self._hash(key), (key, value))
    
    def __setitem__(self, key, value):
        if self.size + 1 > len(self.data) // 1.5:
            self._resize()
        if self._insert(self._hash(key), (key, value)):  
            self.size += 1
    
    def __getitem__(self, key):
        index = self._hash(key)
        if self.data[index] is not None:
            for key_, value in self.data[index]:
                if key_ == key:
                    return value
        raise KeyError()

### Сравнение алгоритмов


### Задание 3
Расставить на стандартной 64-клеточной шахматной доске 8 ферзей так, чтобы ни один из них не находился под боем другого». Подразумевается, что ферзь бьёт все клетки, расположенные по вертикалям, горизонталям и обеим диагоналям Написать программу, которая находит хотя бы один способ решения задач.

##### ИЛИ

Заполнить матрицу размером 8 х 8 нулями и единицами таким образом, чтобы сумма всех элементов матрицы была равна 8, при этом сумма элементов ни в одном столбце, строке или диагональном ряде матрицы не превышала единицы


In [76]:
def queensproblem(rows, columns):
    solutions = [[]]
    for row in range(rows):
        solutions = add_one_queen(row, columns, solutions)
    return solutions

def add_one_queen(new_row, columns, prev_solutions):
    return [solution + [new_column]
            for solution in prev_solutions
            for new_column in range(columns)
            if no_conflict(new_row, new_column, solution)]

def no_conflict(new_row, new_column, solution):
    return all(solution[row]       != new_column           and
               solution[row] + row != new_column + new_row and
               solution[row] - row != new_column - new_row
               for row in range(new_row))

print(solution)

[7, 3, 0, 2, 5, 1, 6, 4]
